In [1]:
from class_definitions import *

Wines = []
for (dirpath,dirnames,filenames) in walk('Wines/'):
    if 'name-search' in dirpath:
        for filename in filenames:
            pkl = joblib.load(dirpath+'/'+filename)
            if pkl.__class__ == Wine:
                Wines.append(pkl)

vivino_wines = []
for w in Wines:
    vivino_wines.extend(w.vivino)

vivino_ids = [vw['wine_link'].split('/')[-1] for vw in vivino_wines]


In [2]:
import os
import sqlalchemy
from sqlalchemy import create_engine
class MyDatabase:
    # http://docs.sqlalchemy.org/en/latest/core/engines.html
    """
    Custom class for instantiating a SQL Alchemy connection. Configured here for SQLite, but intended to be flexible.
    Credit to Medium user Mahmud Ahsan:
    https://medium.com/@mahmudahsan/how-to-use-python-sqlite3-using-sqlalchemy-158f9c54eb32
    """
    DB_ENGINE = {
       'sqlite': 'sqlite:////{DB}'
    }

    # Main DB Connection Ref Obj
    db_engine = None
    def __init__(self, dbtype, 
                 username='', password='', 
                 dbname='',path=os.getcwd()+'/'):
        dbtype = dbtype.lower()
        if dbtype in self.DB_ENGINE.keys():
            engine_url = self.DB_ENGINE[dbtype].format(DB=path+dbname)
            self.db_engine = create_engine(engine_url)
            print(self.db_engine)
        else:
            print("DBType is not found in DB_ENGINE")

In [3]:
db = MyDatabase(dbtype='sqlite',dbname='vivino_db/vivino_wines.db')

Engine(sqlite://///Users/schlinkertc/code/wine_project/wine/vivino_db/vivino_wines.db)


In [4]:
vintages_df = pd.read_sql_table('vintages',db.db_engine)

In [58]:
wines_df = pd.read_sql('select * from wines',db.db_engine)

### Fuzzy matching from KCW to vivino

In [12]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [63]:
vivino_vintages = vintages_df['name'].to_list()

In [64]:
wine = Wines[0]

In [65]:
wine

Wine: Zweigelt 2016 Bauer, Source: name-search

In [66]:
matches = [(x,fuzz.token_sort_ratio(wine.name,x)) for x in vivino_vintages]
sorted(matches,key=lambda x: x[1],reverse=True)[:5]


[('Stefan Bauer Zweigelt', 70),
 ('Norbert Bauer Zweigelt', 68),
 ('Christoph Bauer Zweigelt', 65),
 ('Anton Bauer Zweigelt Alte Reben', 60),
 ('Familie Bauer Zweigelt Klassik', 57)]

In [94]:
vintages_df[vintages_df['name']=='Stefan Bauer Zweigelt']

,index,id,seo_name,name,statistics_status,statistics_ratings_count,statistics_ratings_average,statistics_labels_count,statistics_reviews_count,wine_id,...,ranking_winery_total,ranking_winery_rank,ranking_global_description,ranking_global_total,ranking_global_rank,wine_facts_alcohol,wine_facts_contains_added_sulfites,wine_facts_contains_milk_allergens,wine_facts_contains_egg_allergens,wine_facts_certified_biodynamic
29,29,4470482,stefan-bauer-zweigelt-uv,Stefan Bauer Zweigelt,Normal,26,3.5,4,1,2237206,...,82.0,23.0,Global Rank,7183598.0,2017714.0,NaN,None,None,None,None


In [95]:
wines_df[wines_df['id']==2237206]

,index,id,name,seo_name,type_id,is_natural,region_id,review_status,winery_id,statistics_status,...,statistics_labels_count,statistics_vintages_count,hidden,description,non_vintage,alcohol,created_at,is_first_wine,style_id,sweetness_id
29,29,2237206,Zweigelt,zweigelt,1,0,869.0,2,42667.0,Normal,...,112,12,0,,0,13.0,2014-03-09T08:55:09Z,0,255.0,NaN


In [96]:
f = []
for dirpath,directory,filenames in walk("Wines/vivino_tastingNotes/"):
    f.extend(filenames)
    break

In [99]:
str(4470482) in f

True

In [100]:
joblib.load("Wines/vivino_tastingNotes//4470482")

{'4470482': []}

### WineSearcher Errors 

No result for wine-searcher, but one result for vivino. 
Re-instantiate from the vivino link for wine-searcher match?

In [82]:
wineSearcher_noResults = []
for W in Wines:
    if W.wineSearcher != '403':
        if len(W.wineSearcher.keys())==2:
            wineSearcher_noResults.append(W)

ws_errors = [(W.name,len(W.vivino)) for W in wineSearcher_noResults]

In [68]:
result = vivino_results[0]

In [73]:
{result['vintage']['year'] for result in vivino_results}

{'U.V.'}

In [101]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [128]:
import time
def selenium_scrape(wine_id):
    url = f"https://vivino.com/wines/{str(wine_id)}"
    
    chrome_driver_path = "/Users/schlinkertc/code/chromedriver"
    driver = webdriver.Chrome(executable_path=chrome_driver_path)
    
    driver.get(url)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 1200)")
    driver.execute_script("window.scrollTo(0, 1800)")
    time.sleep(2)
    master_wine_page = driver.find_element_by_id('master-wine-page-app')
    
    driver.set_window_size(width=742,height=983)
    a_tags = driver.find_elements_by_tag_name('a')
    
    for a in a_tags:
        a_class = a.get_attribute('class')
        if 'toggleShowAll' in a_class:
            show_all = a
            break

    a.click()
#     text = master_wine_page.text
#     notes = [x for x in text.split('\n') if 'mentions of' in x]
    
    svgs = master_wine_page.find_elements_by_tag_name('svg')

    tasteNotes = []
    for s in svgs:
        try:
            svg_class = s.get_attribute('class')
            if 'tasteNote' in svg_class:
                tasteNotes.append(s)
        except:
            continue 

    note_texts = []
    notes = []
    for i in range(len(tasteNotes)):
        tasteNotes[i].click()
        time.sleep(5)
        spans = driver.find_elements_by_tag_name('span')
        for span in spans:
            try:
                span_class = span.get_attribute('class')
                if 'tasteReviews__capitalizedTastes' in span_class:
                    note_texts.append(span.text)
            except:
                continue
        divs = driver.find_elements_by_tag_name('div')
        
        
        for div in divs:
            try:
                div_class = div.get_attribute('class')
                if 'noteTag__name' in div_class:
                    note_tags.append({'index':i,'tag':div.text})
            except:
                continue
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        
      ## This is to get the indicator bars. not ready yet  
#     spans = [x for x in master_wine_page.find_elements_by_tag_name('span')]
#     styles = []
#     for span in spans:
#         try:
#             style = span.get_attribute('style')
#             if style.startswith('width'):
#                 styles.append(style)
#         except:
#             continue
    driver.quit()
    indexed_notes = [{'index':n,'note':note} for n,note in enumerate(notes)]
    
    for tag in note_tags:
        for note in indexed_notes:
            if note['index']==tag['index']:
                tag['note']=note['note']
                
    return {wine_id:note_tags}

In [129]:
selenium_scrape(4470482)

{4470482: [{'index': 0,
   'tag': 'blackcurrant',
   'note': '2 mentions of black fruit notes'},
  {'index': 0, 'tag': 'plum', 'note': '2 mentions of black fruit notes'},
  {'index': 1, 'tag': 'prune', 'note': '1 mentions of dried fruit notes'},
  {'index': 2, 'tag': 'violet', 'note': '1 mentions of floral notes'},
  {'index': 3, 'tag': 'chard'},
  {'index': 3, 'tag': 'red beet'},
  {'index': 4, 'tag': 'chocolate'}]}

In [ ]:
https://vivino.com/wines/4470482